# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 26/02/2024

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import concurrent.futures
import timeit
import glob
import pprint
from shutil import rmtree
import time
from functools import partial

import matplotlib
%matplotlib ipympl

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import ImageD11.columnfile
from ImageD11.sinograms import properties, roi_iradon, dataset
from ImageD11.sinograms.sinogram import GrainSinogram, build_slice_arrays, write_slice_recon, read_slice_recon, write_h5, read_h5, write_pbp_strain
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.grain import grain

import ImageD11.nbGui.nb_utils as utils


In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)
   
sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
samples_dict = {}

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_100um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]

for sample in sample_list:
    all_dset_folders_for_sample = os.listdir(os.path.join(rawdata_path, sample))
    dsets_list = []
    for folder in all_dset_folders_for_sample:
        if dset_prefix in folder:
            dset_name = folder.split(f"{sample}_")[1]
            if dset_name not in skips_dict[sample]:
                dsets_list.append(dset_name)

    samples_dict[sample] = dsets_list
    
# manually override:

samples_dict = {"FeAu_0p5_tR_nscope": ["top_150um", "top_200um", "top_250um"]}

In [ ]:
# pick a specific sample to continue

sample = 'FeAu_0p5_tR_nscope'

# pick the name of the H5 group to import the TensorMap from

tmap_h5group = 'TensorMap_YSZ_refined'

from collections import OrderedDict

ds_dict = OrderedDict()

for dataset in samples_dict[sample]:
    # read the ds
    dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
    ds = ImageD11.sinograms.dataset.load(dset_path)
    
    # read the tensormaps
    ds.tensor_map = TensorMap.from_h5(ds.grainsfile, h5group=tmap_h5group)
    ds_dict[dataset] = ds

In [ ]:
# in this case, first scan  has the Z motor at its lowest value
# you should double-check this for your data!
# this means we're scanning the highest region on the sample first
# then moving down in the sample as we increase the Z number
# therefore we want the combined tensormaps in reverse order

# choose the z step for your data
# in our case we assume it's the same as the y step
# this may not be true for you!!!

zstep = ds.ystep

combined_tensormap = TensorMap.from_stack([ds.tensor_map for ds in list(ds_dict.values())][::-1], zstep=zstep)

In [ ]:
# compute and plot strains for Z slice 0

fig, axs = plt.subplots(3,3, sharex=True, sharey=True, layout='constrained')
for i in range(3):
    for j in range(3):
        axs[i,j].imshow(combined_tensormap.eps_sample[0, ..., i, j])
        axs[i,j].set_title(f'eps_{i+1}{j+1}')
plt.show()

In [ ]:
# export to file

combined_tensormap.to_h5('combined.h5')
combined_tensormap.to_paraview('combined.h5')